In [1]:
!mkdir dataset
!cd dataset
!git clone https://huggingface.co/datasets/tatsu-lab/alpaca
!git clone https://huggingface.co/datasets/databricks/databricks-dolly-15k
!git clone https://huggingface.co/datasets/OpenAssistant/oasst1


Cloning into 'alpaca'...
remote: Enumerating objects: 26, done.
remote: Total 26 (delta 0), reused 0 (delta 0), pack-reused 26 (from 1)
Unpacking objects: 100% (26/26), 6.07 KiB | 690.00 KiB/s, done.
Cloning into 'databricks-dolly-15k'...
remote: Enumerating objects: 36, done.
remote: Total 36 (delta 0), reused 0 (delta 0), pack-reused 36 (from 1)
Unpacking objects: 100% (36/36), 8.30 KiB | 849.00 KiB/s, done.
Cloning into 'oasst1'...
remote: Enumerating objects: 144, done.
remote: Total 144 (delta 0), reused 0 (delta 0), pack-reused 144 (from 1)
Receiving objects: 100% (144/144), 25.54 KiB | 12.77 MiB/s, done.
Resolving deltas: 100% (78/78), done.
Filtering content: 100% (8/8), 220.76 MiB | 122.00 MiB/s, done.


In [ ]:
import wandb

wandb.login(key="your-api-key")

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

True

In [3]:
import re
import torch; v = re.match(r"[0-9\\.]{3,}", str(torch.__version__)).group(0)
xformers = "xformers==" + ("0.0.32.post2" if v == "2.8.0" else "0.0.29.post3")
!pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
!pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" "huggingface_hub>=0.34.0" hf_transfer
!pip install --no-deps unsloth
!pip install transformers==4.56.2
!pip install --no-deps trl==0.22.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 MB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.4/270.4 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 11.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.9.0
    Uninstalling fsspec-2025.9.0:
      Successfully uninstalled fsspec-2025.9.0
  Attempting uninstall: dill
    Found existing installation: dill 0.4.0
    Uninstalling dill-0.4.0:
      Successfully uninstalled dill-0.4.0
  Attempting uninstall: datasets
    Found existing installation: datasets 4.1.1
    Uninstalling datasets-4.1.1:
      Successfully u

In [4]:
import json
import os

import numpy as np
import pandas as pd

# Not: argparse artık kullanılmadığı için import edilmiyor.

# Artık komut satırı argümanlarını kullanmayacağımız için
# data_args() fonksiyonu kaldırılmıştır.

def split_to_jsonl(data, output_dir, validation_ratio, testing_ratio):
    print("Preprocessing data to NeMo_SFT jsonl format...")
    output_path_tra = os.path.join(output_dir, "training.jsonl")
    output_path_val = os.path.join(output_dir, "validation.jsonl")
    output_path_tst = os.path.join(output_dir, "testing.jsonl")

    data_ct = len(data)
    # val_threshold ve test_threshold arasındaki indis sıralamasını düzelttik:
    # İlk önce test kümesini ayırıyoruz (val_threshold'a kadar olan kısım test oluyor).
    test_threshold = int(data_ct * testing_ratio)
    val_threshold = int(data_ct * validation_ratio) + test_threshold # testten sonra başlayan kısım

    with open(output_path_val, "w") as g, open(output_path_tst, "w") as h, open(output_path_tra, "w") as i:
        for index, item in data.iterrows():
            # item'daki alanların varlığını ve tipini kontrol ediyoruz
            context = str(item.get("context", "")).strip()
            
            if context != "":
                # Randomize context and instruction order.
                context_first = np.random.randint(0, 2) == 0
                instruction = str(item["instruction"]).strip()
                assert instruction != ""

                if context_first:
                    input_text = f"{context}\n\n{instruction}"
                else:
                    input_text = f"{instruction}\n\n{context}"
                
                output_text = item["response"]
            else:
                input_text = item["instruction"]
                output_text = item["response"]
                
            # 'input' ve 'output' adlarını değiştirdik, Python'da 'input' yerleşik bir fonksiyon adıdır.

            # write to jsonl file according to index
            if index < test_threshold:
                # İlk dilim: Test
                h.write(json.dumps({"input": input_text, "output": output_text}) + "\n")
            elif index < test_threshold + val_threshold:
                # İkinci dilim: Validation (Doğrulama)
                g.write(json.dumps({"input": input_text, "output": output_text}) + "\n")
            else:
                # Kalanı: Training (Eğitim)
                i.write(json.dumps({"input": input_text, "output": output_text}) + "\n")
                
    print(f"{index + 1} Data was successfully preprocessed and saved to {output_dir}.")


def main():
    # --- ARGÜMANLARIN YERİNE SABİT DEĞERLER (NOTEBOOK İÇİN) ---
    
    # Kendi değerlerinizi buraya girin:
    PATH_TO_TRAIN = "/kaggle/working/databricks-dolly-15k/databricks-dolly-15k.jsonl" # training_file değeri
    OUTPUT_DIR = "/kaggle/working/dataset/dolly"                                   # output_dir değeri
    
    # İsteğe bağlı (gerekirse None yapın):
    PATH_TO_VAL = None                                                             # validation_file değeri
    
    # Oranlar:
    VAL_RATIO = 0.1                                                                # validation_ratio
    TEST_RATIO = 0.1                                                               # testing_ratio

    # -----------------------------------------------------------
    
    # load training data
    train = pd.read_json(PATH_TO_TRAIN, lines=True)
    
    # load validation data if provided and append to training data
    if PATH_TO_VAL:
        val = pd.read_json(PATH_TO_VAL, lines=True)
        train = pd.concat([train, val])
        
    # randomize the order of the data
    data_full = train.sample(frac=1, random_state=0).reset_index(drop=True)
    
    # split data into training, validation and testing
    if not os.path.exists(OUTPUT_DIR):
        os.makedirs(OUTPUT_DIR)
        
    split_to_jsonl(data_full, OUTPUT_DIR, VAL_RATIO, TEST_RATIO)


if __name__ == "__main__":
    main()

Preprocessing data to NeMo_SFT jsonl format...
15011 Data was successfully preprocessed and saved to /kaggle/working/dataset/dolly.


In [5]:
# --- HÜCRE 1: Kurulum ve Gerekli Kütüphaneler ---
# Bu kurulum, Unsloth'un en hızlı 4-bit/QLoRA modunu ve gerekli paketleri sağlar.

import os
import re
import random
import shutil
import datasets
import numpy as np
import torch
from unsloth import FastLanguageModel
from unsloth.chat_templates import get_chat_template
from trl import SFTConfig, SFTTrainer
from transformers import DataCollatorForSeq2Seq
from peft import LoraConfig, get_peft_model, get_peft_model_state_dict, set_peft_model_state_dict, utils
from transformers import trainer_utils


# set deterministic seed for reproducibility (Kullanıcı tarafından belirtilen tohum değerleri)
torch.manual_seed(0)
random.seed(0)
np.random.seed(0)

# --- HÜCRE 2: Model ve LoRA Kurulumu ---
max_seq_length = 1024 # Kullanıcının SFTConfig'de belirttiği maksimum uzunluk
dtype = None # None for auto detection. 
load_in_4bit = True # 4bit kuantizasyon (QLoRA)

# Kullanıcının hedeflediği model: Llama-3.2-1B
model_name = "unsloth/Llama-3.2-1B-Instruct-bnb-4bit"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # Gated modeller için
)

# LoRA adaptörlerini ekleyin (Kullanıcının r=64, lora_alpha=16, lora_dropout=0.1 ayarlarıyla)
# Unsloth'un optimize edilmiş LoRA katmanları kullanılır.
model = FastLanguageModel.get_peft_model(
    model,
    r = 64, # Kullanıcının parametresi
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16, # Kullanıcının parametresi
    lora_dropout = 0.1, # Kullanıcının parametresi
    bias = "none", # Unsloth için optimize
    use_gradient_checkpointing = "unsloth", # Unsloth'un optimize edilmiş VRAM kullanımı
    random_state = 0, # Kullanıcının tohum değeri
    use_rslora = False,
    loftq_config = None, 
)

# Gradient Checkpointing ayarı: model.config.use_cache = False (Unsloth tarafından zaten yönetilir)
# Ayrıca QLoRA için input grad'lerin etkinleştirilmesi: (Unsloth tarafından zaten yapılır)
if hasattr(model, "enable_input_require_grads"):
     model.enable_input_require_grads()
else:
     def make_inputs_require_grad(module, input, output):
          output.requires_grad_(True)
     model.get_input_embeddings().register_forward_hook(make_inputs_require_grad)


# --- HÜCRE 3: Veri Ön İşleme ve Yükleme ---
# Kullanıcının özel format_instruction mantığını uygulamak için bir fonksiyon tanımlanır.
def formatting_prompts_func(examples):
    # Kullanıcının formatı: "### Instruction: ... ### Input:{} ### Response: {}"
    texts = [
        f"### Instruction: Generate Output according to the information and question given by Input. ### Input:{inp} ### Response: {out}"
        for inp, out in zip(examples['input'], examples['output'])
    ]
    return { "text" : texts }

# Veri yolları (Kullanıcının script'inden gelen değerler)
data_path_train = "./dataset/dolly/training.jsonl"
data_path_valid = "./dataset/dolly/validation.jsonl"

try:
    # Kullanıcının ön işleme scripti tarafından oluşturulan dosyaları yükler
    dataset_train = datasets.load_dataset("json", data_files=data_path_train, split="train")
    dataset_valid = datasets.load_dataset("json", data_files=data_path_valid, split="train")
except FileNotFoundError:
    print(f"UYARI: {data_path_train} veya {data_path_valid} bulunamadı.")
    print("DEMO amaçlı sahte bir veri kümesi oluşturuluyor.")
    
    # Dosya bulunamazsa demo amaçlı bir yapı oluşturulur
    dummy_data = {
        'input': ["Explain Unsloth.", "What is the Fibonacci sequence?"],
        'output': ["Unsloth is a library for fast and memory-efficient fine-tuning of large language models, offering significant speed and VRAM reductions.", "The Fibonacci sequence is a series of numbers where each number is the sum of the two preceding ones."],
    }
    dataset_train = datasets.Dataset.from_dict(dummy_data)
    dataset_valid = datasets.Dataset.from_dict(dummy_data)

# Özel formatlama fonksiyonunu uygulamak için map edilir
dataset_train = dataset_train.map(formatting_prompts_func, batched = True,)
dataset_valid = dataset_valid.map(formatting_prompts_func, batched = True,)

print(f"Dataset size: training {len(dataset_train)}, validation {len(dataset_valid)}")


# --- HÜCRE 4: Trainer Kurulumu ve Tekrarlı Eğitim Mantığı ---

output_path = "./workspace_centralized/llama-3.2-1b-dolly-sft-iter"

# Eğitim argümanları (max_steps kaldırıldı)
train_args = SFTConfig(
    output_dir=output_path,
    num_train_epochs=1, # Her turda bu 1 epoch tamamlanmaya çalışılacak
    per_device_train_batch_size=2, # T4 için optimize edildi
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=4, # T4 için optimize edildi
    optim="adamw_8bit", # Unsloth'un optimize edilmiş optimizer'ı
    logging_steps=10, # Hızlı notebook çıktısı için
    save_strategy="steps", 
    save_steps=50,
    learning_rate=5e-4, # Kullanıcının parametresi
    lr_scheduler_type="constant", # Kullanıcının parametresi
    seed=0,
    # max_steps kaldırıldı. Eğitim num_train_epochs=1'e göre çalışacak.
    max_seq_length=max_seq_length,
    dataset_text_field = "text",
    #report_to = "none", # Uyarıları önlemek için eklendi
    disable_tqdm = False,
    # Diğer kullanıcı parametreleri:
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    save_total_limit=2,
    save_safetensors=False,
)

# Trainer başlatma
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset_train,
    eval_dataset=dataset_valid,
    args=train_args,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer, model = model),
    # Unsloth, formatlanmış metin alanını otomatik olarak kullanır
)

# Tekrarlı Eğitim Mantığı için Başlangıç Ağırlıklarının Kaydedilmesi
# Bu, her döngüde modelin aynı başlangıç ağırlıklarından başlamasını sağlamak için gereklidir.
os.makedirs(output_path, exist_ok=True)
initial_model_path = os.path.join(output_path, "pytorch_model_initial.pth")

# PEFT/LoRA adaptör ağırlıklarını kaydet
params = get_peft_model_state_dict(model)
torch.save(params, initial_model_path)
print(f"Başlangıç PEFT model ağırlıkları kaydedildi: {initial_model_path}")

# --- TEKRARLI EĞİTİM DÖNGÜSÜ ---
num_rounds = 3 # Tur sayısı (Kullanıcının isteği)

for curr_round in range(num_rounds):
    print("\n" + "="*80)
    print(f"BAŞLANGIÇ EĞİTİM TURU {curr_round + 1} / {num_rounds}")
    print("="*80)

    # 1. Her yeni tur için modeli başlangıç ağırlıklarına sıfırla
    print(f"DEBUG: Tur {curr_round + 1}. Başlangıç ağırlıkları yükleniyor...")
    state_dict_replace = torch.load(initial_model_path)
    set_peft_model_state_dict(trainer.model, state_dict_replace)
    print("PEFT model ağırlıkları başlangıç durumuna sıfırlandı.")
    
    # 2. Eğitimi Başlat
    if curr_round == 0:
        # İlk tur: Sıfırdan başlat
        print(f"DEBUG: İlk tur. trainer.train() sıfırdan başlıyor. Num Epochs: {trainer.args.num_train_epochs}")
        trainer.train()
    else:
        # Sonraki turlar: En son checkpoint'i başlangıç ağırlıklarıyla değiştir ve devam et
        resume_from_checkpoint_folder = trainer_utils.get_last_checkpoint(trainer.args.output_dir)
        
        print(f"DEBUG: Devam turu. Bulunan en son checkpoint klasörü: {resume_from_checkpoint_folder}")

        if resume_from_checkpoint_folder is None:
             print("HATA: Devam edilecek bir checkpoint bulunamadı. Döngüden çıkılıyor.")
             break
        
        try:
            # Checkpoint içindeki PEFT ağırlık dosyasını (adapter_model.safetensors veya pytorch_model.bin)
            # başlangıç ağırlıklarıyla değiştiriyoruz.
            resume_model_file_path = os.path.join(resume_from_checkpoint_folder, utils.WEIGHTS_NAME)
            print(f"DEBUG: Checkpoint içinde değiştirilecek dosya yolu: {resume_model_file_path}")
            
            # Kullanıcının mantığına göre direkt torch.save ile üzerine yazma
            torch.save(state_dict_replace, resume_model_file_path)
            print(f"Checkpoint ağırlıkları ({resume_model_file_path}) başlangıç ağırlıklarıyla DEĞİŞTİRİLDİ.")
            
        except Exception as e:
            print(f"KRİTİK HATA: Checkpoint değiştirme sırasında dosya hatası oluştu: {e}") 
            print(f"Tahmini PEFT ağırlık dosya adı: {utils.WEIGHTS_NAME}")
            print("Lütfen checkpoint klasörünün içeriğini (örneğin 'checkpoint-50') kontrol edin.")
            break
        
        # Orijinal scriptteki mantık: num_train_epochs'u 1 artır ve eğitime devam et
        trainer.args.num_train_epochs += 1
        print(f"DEBUG: num_train_epochs artırıldı: {trainer.args.num_train_epochs}. trainer.train(resume_from_checkpoint=True) ile devam ediliyor.")
        trainer.train(resume_from_checkpoint=True)
        
# Eğitim tamamlandı
print("\n" + "="*80)
print(f"Tekrarlı Eğitim Döngüsü Tamamlandı ({num_rounds} tur).")
print("="*80)


# --- HÜCRE 5: Çıkarım (Inference) ve Kaydetme ---

# Çıkarım (Inference) için model optimize edilir
FastLanguageModel.for_inference(model)

# Örnek çıkarım
custom_prompt = "Generate Output according to the information and question given by Input. ### Input:What are the three main types of rock? ### Response:"
inputs = tokenizer(
    [custom_prompt], 
    return_tensors = "pt",
    padding=True,
    truncation=True,
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
print("\n" + "="*80)
print("Örnek Çıkarım:")
print("="*80)
_ = model.generate(
    **inputs, 
    streamer = text_streamer, 
    max_new_tokens = 64,
    use_cache = True, 
    temperature = 0.7, # Inference ayarları
    min_p = 0.1, # Inference ayarları
)
print("\n" + "="*80)

# LoRA adaptörlerini kaydetme (En son tur ağırlıkları)
model_save_path = "lora_model_final"
model.save_pretrained(model_save_path) 
tokenizer.save_pretrained(model_save_path)
print(f"Son LoRA adaptörleri yerel olarak kaydedildi: {model_save_path}")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-10-25 23:45:55.512163: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761435955.900264      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761435956.027913      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.10.9: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/1.03G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

...

Step,Training Loss
1320,2.705100
1330,2.118500
1340,2.006600
1350,1.954300
1360,1.943400
1370,1.898500
1380,2.135000
1390,2.057500
1400,1.955800
1410,2.064800



BAŞLANGIÇ EĞİTİM TURU 3 / 3
DEBUG: Tur 3. Başlangıç ağırlıkları yükleniyor...
PEFT model ağırlıkları başlangıç durumuna sıfırlandı.
DEBUG: Devam turu. Bulunan en son checkpoint klasörü: ./workspace_centralized/llama-3.2-1b-dolly-sft-iter/checkpoint-2628
DEBUG: Checkpoint içinde değiştirilecek dosya yolu: ./workspace_centralized/llama-3.2-1b-dolly-sft-iter/checkpoint-2628/adapter_model.bin
Checkpoint ağırlıkları (./workspace_centralized/llama-3.2-1b-dolly-sft-iter/checkpoint-2628/adapter_model.bin) başlangıç ağırlıklarıyla DEĞİŞTİRİLDİ.
DEBUG: num_train_epochs artırıldı: 3. trainer.train(resume_from_checkpoint=True) ile devam ediliyor.


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 10,508 | Num Epochs = 3 | Total steps = 3,942
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 45,088,768 of 1,280,903,168 (3.52% trained)


Step,Training Loss
2630,2.853300
2640,2.268800
2650,2.158400
2660,1.981700
2670,1.975100
2680,2.049000
2690,1.930000
2700,2.046200
2710,1.893600
2720,1.863200



Tekrarlı Eğitim Döngüsü Tamamlandı (3 tur).

Örnek Çıkarım:
 The three main types of rock are igneous, sedimentary and metamorphic rocks. Igneous rocks are formed by the cooling of magma, sedimentary rocks are formed by the accumulation of layers of sediments and metamorphic rocks are formed by the deformation of existing rocks. There are many other types of rocks as well

Son LoRA adaptörleri yerel olarak kaydedildi: lora_model_final


In [6]:
# --- HÜCRE 5: Çıkarım (Inference) ve Kaydetme ---

# Çıkarım (Inference) için model optimize edilir
FastLanguageModel.for_inference(model)

# Örnek çıkarım
custom_prompt = "Generate Output according to the information and question given by Input. ### Input:What are the three main types of rock? ### Response:"
inputs = tokenizer(
    [custom_prompt], 
    return_tensors = "pt",
    padding=True,
    truncation=True,
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
print("\n" + "="*80)
print("Örnek Çıkarım:")
print("="*80)
_ = model.generate(
    **inputs, 
    streamer = text_streamer, 
    max_new_tokens = 64,
    use_cache = True, 
    temperature = 0.7, # Inference ayarları
    min_p = 0.1, # Inference ayarları
)
print("\n" + "="*80)

# LoRA adaptörlerini kaydetme (En son tur ağırlıkları)
model_save_path = "lora_model_final"
model.save_pretrained(model_save_path) 
tokenizer.save_pretrained(model_save_path)
print(f"Son LoRA adaptörleri yerel olarak kaydedildi: {model_save_path}")

# Modeli float16 veya GGUF formatına kaydetme (Seçenekler)
# if False: model.save_pretrained_merged("model_16bit", tokenizer, save_method = "merged_16bit",)
# if False: model.save_pretrained_gguf("model_gguf", tokenizer, quantization_method = "q4_k_m")


Örnek Çıkarım:
 The three main types of rock are igneous, sedimentary, and metamorphic. Each type of rock is formed from the solidification of magma or lava. Sedimentary rocks are formed from the accumulation of sediments. Metamorphic rocks are formed from the transformation of existing rocks through heat and pressure. There are also

Son LoRA adaptörleri yerel olarak kaydedildi: lora_model_final
